In [5]:
import numpy as np
import tensorflow as tf
import sleap
from pathlib import Path
from sleap.nn.config import UNetConfig


In [6]:
dataset = r"C:\Users\neurogears\Desktop\SLEAPNetwork_v1\justTracks.pkg.slp" # load the dataset with tracks
skeleton_path = r"C:\Users\neurogears\Desktop\SLEAPNetwork_v1\skeleton.json"
videos = ["CameraTop_1904-01-07T00-00-00.avi"] # videos just in case, should not be needed as the dataset already has the path to the videos
run_name = "multiclass.topdown"
runs_folder = r"C:\Users\neurogears\Desktop\Sleap_demo\models"

In [7]:
skeleton = sleap.Skeleton.load_json(skeleton_path) # load the skeleton
labels = sleap.load_file(dataset) # load the label data
labels.tracks

[Track(spawned_on=0, name='1'), Track(spawned_on=0, name='2')]

In [8]:
skeleton = sleap.Skeleton.load_json(skeleton_path) # load the skeleton
labels = sleap.load_file(dataset) # load the label data

cfg = sleap.TrainingJobConfig.load_json(r"C:\Users\neurogears\Documents\git\SLEAP\sleap\sleap\training_profiles\baseline_large_rf.topdown.json") # start with a default settings
cfg.model.heads.centered_instance = None # Must disable the original head since only one head is allowed per full model

#cfg = sleap.TrainingJobConfig()
cfg.data.instance_cropping.center_on_part = "center"
cfg.data.instance_cropping.crop_size = 96

cfg.data.labels.skeletons = [skeleton] # load skeleton
cfg.data.labels.training_labels = dataset

cfg.model.backbone.unet = UNetConfig() # select the backbone
#configure the backbone
cfg.model.backbone.unet.max_stride = 32
cfg.model.backbone.unet.stem_stride = None
cfg.model.backbone.unet.filters = 24
cfg.model.backbone.unet.filters_rate = 1.5
cfg.model.backbone.unet.middle_block = True
cfg.model.backbone.unet.up_interpolate = True


#optimizer settings
cfg.optimization.preload_data = True
cfg.optimization.batches_per_epoch = None
cfg.optimization.min_batches_per_epoch = 200
cfg.optimization.val_batches_per_epoch = 100
cfg.optimization.min_val_batches_per_epoch = 10
cfg.optimization.batch_size = 4
cfg.optimization.epochs = 600

cfg.optimization.initial_learning_rate = 0.0001
cfg.optimization.learning_rate_schedule.reduce_on_plateau = True
cfg.optimization.learning_rate_schedule.reduction_factor = 0.1
cfg.optimization.learning_rate_schedule.plateau_min_delta = 1e-08
cfg.optimization.learning_rate_schedule.plateau_patience = 20
cfg.optimization.learning_rate_schedule.plateau_cooldown = 3
cfg.optimization.learning_rate_schedule.min_learning_rate = 1e-08


cfg.optimization.early_stopping.stop_training_on_plateau = True
cfg.optimization.early_stopping.plateau_min_delta = 1e-08
cfg.optimization.early_stopping.plateau_patience = 30

cfg.optimization.online_shuffling = True
cfg.optimization.shuffle_buffer_size = 128
cfg.optimization.prefetch = True


cfg.optimization.augmentation_config.scale = True
cfg.optimization.augmentation_config.scale_min = 0.9
cfg.optimization.augmentation_config.scale_max = 1.1


#Configure outputs
cfg.outputs.run_name = run_name
cfg.outputs.save_outputs = True
cfg.outputs.runs_folder = runs_folder
cfg.outputs.save_visualizations = True
cfg.outputs.checkpointing.initial_model = True
cfg.outputs.checkpointing.best_model = True



In [9]:
confmaps=sleap.nn.config.model.CenteredInstanceConfmapsHeadConfig(
    anchor_part=None,
    part_names = [part.name for part in skeleton.nodes],
    sigma = 2.5, output_stride = 4, loss_weight = 100, offset_refinement = False
    )#increase confmaps loss_weight

class_vectors=sleap.nn.config.ClassVectorsHeadConfig(
    classes = ['A', 'B'],
    output_stride=4,
    num_fc_layers=3,
    num_fc_units=256,
    global_pool=True,
    loss_weight = 0.5)

cfg.model.heads.multi_class_topdown = sleap.nn.config.MultiClassTopDownConfig(
    confmaps=confmaps,
    class_vectors=class_vectors,
)


In [10]:
trainer = sleap.nn.training.TopDownMultiClassModelTrainer.from_config(
    cfg, training_labels=labels
)

INFO:sleap.nn.training:Creating training and validation splits from validation fraction: 0.1
INFO:sleap.nn.training:  Splits: Training = 79 / Validation = 9.


In [11]:
trainer.setup()
trainer.train()

print(trainer.keras_model.outputs[0].shape)
print(trainer.keras_model.outputs[1].shape)

INFO:sleap.nn.training:Setting up for training...
INFO:sleap.nn.training:Setting up pipeline builders...
INFO:sleap.nn.training:Setting up model...
INFO:sleap.nn.training:Building test pipeline...
INFO:sleap.nn.training:Loaded test example. [5.530s]
INFO:sleap.nn.training:  Input shape: (96, 96, 1)
INFO:sleap.nn.training:Created Keras model.
INFO:sleap.nn.training:  Backbone: UNet(stacks=1, filters=24, filters_rate=1.5, kernel_size=3, stem_kernel_size=7, convs_per_block=2, stem_blocks=0, down_blocks=5, middle_block=True, up_blocks=3, up_interpolate=True, block_contraction=False)
INFO:sleap.nn.training:  Max stride: 32
INFO:sleap.nn.training:  Parameters: 1,791,145
INFO:sleap.nn.training:  Heads: 
INFO:sleap.nn.training:    [0] = CenteredInstanceConfmapsHead(part_names=['Snout', 'TailBase', 'RightEar', 'LeftEar', 'center'], anchor_part=None, sigma=2.5, output_stride=4, loss_weight=100)
INFO:sleap.nn.training:    [1] = ClassVectorsHead(classes=['A', 'B'], num_fc_layers=3, num_fc_units=25

Output()

INFO:sleap.nn.evals:Saved predictions: C:\Users\neurogears\Desktop\SLEAPNetwork_v1\models\multiclass.topdown_1\labels_pr.train.slp
INFO:sleap.nn.evals:Saved metrics: C:\Users\neurogears\Desktop\SLEAPNetwork_v1\models\multiclass.topdown_1\metrics.train.npz
INFO:sleap.nn.evals:OKS mAP: 0.446055


Output()

INFO:sleap.nn.evals:Saved predictions: C:\Users\neurogears\Desktop\SLEAPNetwork_v1\models\multiclass.topdown_1\labels_pr.val.slp
INFO:sleap.nn.evals:Saved metrics: C:\Users\neurogears\Desktop\SLEAPNetwork_v1\models\multiclass.topdown_1\metrics.val.npz
INFO:sleap.nn.evals:OKS mAP: 0.017238
(None, 24, 24, 5)
(None, 2)


In [4]:
from test_run import *
import test_utils
from pathlib import Path
from sleap.nn.peak_finding import find_local_peaks, find_global_peaks, make_centered_bboxes, crop_bboxes
from sleap.nn.data.utils import  describe_tensors

id_model = tf.keras.models.load_model(runs_folder + "\\" + run_name + "\\best_model.h5", compile = False)

#Define the full mode
class TopDownInferenceModel(tf.keras.Model):
    def __init__(
        self,
        this_model,
        td_output_stride,
        **kwargs
    ):
        super().__init__(**kwargs)
        self.this_model = this_model
        self.td_output_stride = td_output_stride


    @tf.function
    def stage2(self, im):
        """Stage 2: Predict pose in each crop"""
        # Preprocessing
        X = tf.cast(im, tf.float32) / 255
        X = tf.image.resize(X, [96, 96])

        # Forward pass
        cms, class_probs = self.this_model(X)

        # Find keypoints in each crop
        #     pts: (n_centroids, n_nodes, 2)
        #     vals: (n_centroids, n_nodes)
        pts, vals = find_global_peaks(cms, threshold=0.2, refinement="integral")

        # Adjust coordinates for output stride
        pts = pts * self.td_output_stride

        return {"instance_peaks": pts, "instance_peak_vals": vals, "class_probabilities": class_probs}

    def call(self, imgs):
        preds = self.stage2(imgs)
        return preds

td_output_stride = 4
td_input_size = [96, 96] #Hardcoded in the network

inference_model = TopDownInferenceModel(id_model,td_output_stride)

preds = inference_model.predict(np.zeros((4, 96, 96, 1), dtype="uint8"))
describe_tensors(preds)

inference_model.save(
    r"C:\Users\neurogears\Desktop\SLEAPNetwork_v1\Bonsai\customNetwork" + "\\" + run_name,
    save_format="tf", save_traces=True)


     instance_peaks: type=ndarray, shape=(4, 5, 2), dtype=float32, device=N/A
 instance_peak_vals: type=ndarray, shape=(4, 5), dtype=float32, device=N/A
class_probabilities: type=ndarray, shape=(4, 2), dtype=float32, device=N/A
INFO:tensorflow:Assets written to: C:\Users\neurogears\Desktop\SLEAPNetwork_v1\Bonsai\customNetwork\multiclass.topdown.2\assets


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2

#Import the network
imported = tf.saved_model.load(r"C:\Users\neurogears\Desktop\SLEAPNetwork_v1\Bonsai\customNetwork" + "\\" + run_name)

model = imported.signatures["serving_default"]
print(model)

full_model = tf.function(lambda x: model(x))

full_model = full_model.get_concrete_function(
    tf.TensorSpec(model.inputs[0].shape, model.inputs[0].dtype))
frozen_func = convert_variables_to_constants_v2(full_model)
frozen_func.graph.as_graph_def()

print("Frozen model inputs: ")
print(frozen_func.inputs)
print("Frozen model outputs: ")
print(frozen_func.outputs)


tf.io.write_graph(graph_or_graph_def=frozen_func.graph,
               logdir=r"C:\Users\neurogears\Desktop\SLEAPNetwork_v1\Bonsai\customNetwork" + "\\" + run_name,
               name="frozen_graph.pb",
               as_text=False)

ConcreteFunction signature_wrapper(*, input_1)
  Args:
    input_1: uint8 Tensor, shape=(None, 96, 96, 1)
  Returns:
    {'class_probabilities': <1>, 'instance_peak_vals': <2>, 'instance_peaks': <3>}
      <1>: float32 Tensor, shape=(None, 2)
      <2>: float32 Tensor, shape=(None, 5)
      <3>: float32 Tensor, shape=(None, 5, 2)
Frozen model inputs: 
[<tf.Tensor 'x:0' shape=(None, 96, 96, 1) dtype=uint8>]
Frozen model outputs: 
[<tf.Tensor 'Identity:0' shape=(None, 2) dtype=float32>, <tf.Tensor 'Identity_1:0' shape=(None, 5) dtype=float32>, <tf.Tensor 'Identity_2:0' shape=(None, 5, 2) dtype=float32>]


'C:\\Users\\neurogears\\Desktop\\SLEAPNetwork_v1\\Bonsai\\customNetwork\\multiclass.topdown\\frozen_graph.pb'